# Modelos Lineares

**DEFINIÇÃO**  
Modelos de regressão onde a variável alvo (*variável independente*) pode ser expressa como uma combinação linear das variáveis preditoras (*variáveis independentes*), na forma:  
$$\hat y(\omega _{i}, x_{i}) = \omega _{0} + \sum _{i=1}^{n} \omega _{i} x_{i}$$

# Mínimos Quadrados Ordinários

**DEFINIÇÃO**  
Métodologia para cálculo dos coeficientes de uma regressão linear a partir da otimização do quadrado dos erros (SQE)

**PRESSUPOSTOS**  
Regressores exógenos: para evitar o viés dos coeficientes  
Homocedasticidade: variância constante  
Observações não correlacionadas: evitar autocorrelação na variável alvo

**DEMONSTRAÇÃO (REGRESSÃO LINEAR SIMPLES)**  
Data uma base de dados com $n$ instâncias do tipo $(x_{i}, y_{i})$, é possível ajustar uma reta do tipo:
$$\hat y(\hat x) = a + b \hat x$$  
A distância entre o i-ésimo ponto e a reta ajustada pelo demolo pode ser calculada a partir da expressão:  
$$u_{i} = \sqrt{(y_{i} - y)^2 + (x_{i} - x)^2}$$  
$$u_{i}^2 = (y_{i} - y)^2 + (x_{i} - x)^2$$  
Utilizando o pressuposto de *regressores exógenos* temos $x_{i} = x$, e portanto o quadrado da distância é dada por:  
$$u_{i}^2 = (y_{i} - y)^2$$  
Sendo $u_{i}$ definido como o resíduo do modelo ajustado. Uma vez que sabemos qual é a equação da reta ajustada, podemos escrever:  
$$u_{i}^2 = (y_{i} - a - b*x_{i})^2$$  
Definimos o método de mínimos quadrados como uma função dos coeficientes capaz de minimizar o quadrado dos resíduos, na forma:  
$$F(u) = \sum _{i}^{n} u_{i}^{2}$$  
$$F(a, b) = \sum _{i}^{n} (y_{i} - a - b*x_{i})^2$$  
Na tentativa de encontrar os pontos críticas dessa função, fazemos a derivada parcial em relação aos coeficientes, na forma:  
$$\frac{\partial F}{\partial a} = 2 \sum _{i}^{n} (y_{i} - a - b*x_{i})$$  
$$\frac{\partial F}{\partial b} = 2 \sum _{i}^{n} x_{i}*(y_{i} - a - b*x_{i})$$  
Com a definição dos pontos críticos, buscamos os valores de mínimo igualando os resultados das derivadas parciais a zero, iniciando pelo resultado da derivada em $a$:  
$$2 \sum _{i}^{n} (y_{i} - a - b*x_{i}) = 0$$  
$$\frac{2 \sum _{i}^{n} (y_{i} - a - b*x_{i})}{2n} = \frac{0}{2n}$$  
$$\bar y - a - b* \bar x = 0$$  
$$\color{red}{a = \bar y - b* \bar x}$$  
Para o resultado da derivada em $b$ e utilizando a expressão destacada:
$$2 \sum _{i}^{n} x_{i}*(y_{i} - a - b*x_{i}) = 0$$  
$$2 \sum _{i}^{n} x_{i}*(y_{i} - \bar y + b* \bar x - b*x_{i}) = 0$$  
$$\sum _{i}^{n} x_{i}*(y_{i} - \bar y) - b* \sum _{i}^{n} x_{i}*(x_{i} - \bar x) = 0$$  
$$\color{red}{b = \frac{\sum _{i}^{n} x_{i}*(y_{i} - \bar y)}{\sum _{i}^{n} x_{i}*(x_{i} - \bar x)} = \frac{\sum _{i}^{n} (x_{i} - \bar x)*(y_{i} - \bar y)}{\sum _{i}^{n} (x_{i} - \bar x)^2}}$$ 

**DEMONSTRAÇÃO (REGRESSÃO LINEAR MÚLTIPLA)**  
Dada a expressão referente a um ajuste linear n-dimensional:  
$$\hat y(\omega _{i}, x_{i}) = \omega _{0} + \sum _{i=1}^{n} \omega _{i} x_{i} + \sum _{i=1}^{n} u_{i}$$  
Pode-se escrever utilizando um formato matricial como:  
$$\begin{bmatrix} y_{1} \\ y_{2} \\ ... \\ y_{j} \end{bmatrix} = \begin{bmatrix} 1 & x_{11} & x_{12} & ... & x_{1j} \\ 1 & x_{21} & x_{22} & ... & x_{2j} \\ ... & ... & ... & ... & ... \\ 1 & x_{i1} & x_{i2} & ... & x_{ij} \end{bmatrix} * \begin{bmatrix} \omega_{1} \\ \omega_{2} \\ ... \\ \omega_{i} \end{bmatrix} + \begin{bmatrix} u_{1} \\ u_{2} \\ ... \\ u_{j} \end{bmatrix}$$  
Ou, na notação reduzida:  
$$y = X*\omega + u$$. 
Seguindo o método dos mínimos quadrados, criando uma função para minimização dos resíduos, temos:  
$$F(u) = \sum _{i}^{n} u_{i}^{2}$$  
$$F(\omega) = (y - X*\omega)^{T}(y - X*\omega)$$  
$$F(\omega) = y^{T}y - y^{T}X\omega - \omega^{T}X^{T}y + \omega^{T}X^{T}X\omega$$  
$$F(\omega) = y^{T}y - 2\omega^{T}X^{T}y + X^{T}X\omega^2$$  
Buscando o ponto crítico da função, realizaremos a derivada em função do coeficiente $\omega$:  
$$\frac{\partial F}{\partial \omega} = - 2X^{T}y + 2X^{T}X\omega$$  
Igualando o resultado da derivada parcial a zero encontramos o valor do coeficiente para obtenção do valor mínimo do quadrado do resíduo, na forma:
$$- 2X^{T}y + 2X^{T}X\omega = 0$$  
$$\color{red}{\omega = (X^{T}X)^{-1}(X^{T}y)}$$